In [ ]:
import pandas as pd

data = [


+,+,0,0,+,+,0,+,0,+,nt,nt,+,0,+,0,0,+,+,+,+,0,+,0,+,+,n/a
+,+,0,0,+,0,+,+,0,+,nt,nt,0,+,0,+,0,0,+,+,+,0,+,0,+,0,n/a
+,0,+,+,0,0,0,+,0,+,nt,nt,0,+,+,+,0,+,+,+,0,+,0,0,+,0,n/a








]


columns = [
    'Spender', 'D', 'C', 'E', 'c', 'e', 'CW', 'K', 'k', 'KpA', 'KpB', 'JsA', 'JsB',
    'FyA', 'FyB', 'Jka', 'Jkb', 'Lea', 'Leb', 'P1', 'M', 'N', 'S', 's',
    'LuA', 'LuB', 'XgA', 'Gen.', 'Spez. Antigen', 'LISS'
]

df = pd.DataFrame(data, columns=columns)

df


################### Noch zu machen ################################################ 02.04.2025
# ID ab 1 anstatt 0, Resultat Tabelle vollständig zeigen mit den übrig geblibenen Antigenen
# id panel ist nut 7 wochen haltbar

# main workloads
############################
# möglichkeiten daten extrahieren zb pdf text extraktor oder manuel eingabe
# interface, dropdown menu für negativ = - , positiv = +/- 1+ 2+ 3+ 4+
# regelwerk testen
# 2 stuffen einbauen zum beispiel mit bestätigung etc nachdem man die reaktion eingegeben hat, sieht man die welche spalten rausgenmomen werden.

# programm oder interface

###################################################

,Spender,D,C,E,c,e,CW,K,k,KpA,...,M,N,S,s,LuA,LuB,XgA,Gen.,Spez. Antigen,LISS
0,251718,+,+,0,0,+,+,0,+,0,...,+,0,+,0,+,+,+,N/A,4+,None
1,456531,+,+,0,0,+,0,+,+,0,...,+,+,0,+,0,+,0,N/A,,4+
2,547641,+,0,+,+,0,0,0,+,0,...,+,0,+,0,0,+,0,N/A,,-
3,5830,0,+,0,+,+,0,0,+,0,...,0,+,+,+,0,+,0,N/A,,4+
4,824734,0,0,+,+,+,0,0,+,0,...,+,0,+,0,0,+,+,N/A,Yt(b+)*,-
5,314242,0,0,0,+,+,0,+,+,0,...,0,+,0,+,0,+,+,N/A,,-
6,803065,0,0,0,+,+,0,0,+,+,...,+,0,0,+,0,+,0,N/A,HLA+*,-
7,399620,+,0,0,+,+,0,0,+,0,...,+,+,0,+,0,+,0,N/A,,-
8,514266,0,0,0,+,+,0,0,+,0,...,+,0,+,0,+,+,+,N/A,,-
9,5885,0,0,0,+,+,0,0,+,0,...,0,+,+,0,0,+,+,N/A,,-


In [9]:

df = pd.read_csv("data.csv")

# 3. Entferne unerwünschte Spalten für die Auswertungslogik
check = df.drop(['Spender', 'Spez. Antigen', 'Gen.'], axis=1)

# 4. Definition der homo-/heterozygoten Antigenpaare
homo_hetero_paar = {
    'C': ['C', 'c', 'CW'],
    'E': ['E', 'e'],
    'K': ['K', 'k'],
    'Kp': ['KpA', 'KpB'],
    'Js': ['JsA', 'JsB'],
    'Fy': ['FyA', 'FyB'],
    'Jk': ['Jka', 'Jkb'],
    'Le': ['Lea', 'Leb'],
    'MNS': ['M', 'N', 'S', 's'],
    'Lu': ['LuA', 'LuB']
}

# 5. Für spätere Regelumsetzung: Definiere einzelne Antigen-Paare und erlaubte heterozygote Streichungen
antigen_paare = [
    ("C", "c"),
    ("E", "e"),
    ("K", "k"),
    ("KpA", "KpB"),
    ("JsA", "JsB"),
    ("FyA", "FyB"),
    ("Jka", "Jkb"),
    ("Lea", "Leb"),
    ("M", "N"),
    ("S", "s"),
    ("LuA", "LuB")
]
erlaubte_hetero_streichung = ["CW", "K", "KpA", "LuA"]

# 6. Spender mit LISS == "-" herausfiltern (Regel 1 & 2)
negative_liss = check[check["LISS"] == "-"].drop(columns=["LISS"])
spender_ids = df.loc[negative_liss.index, "Spender"].reset_index(drop=True)

# 7. Hilfsfunktion zur Zygotie-Bestimmung
def zygotie_status(val1, val2):
    if val1 == "+" and val2 == "+":
        return "hetero"
    elif val1 == "+" or val2 == "+":
        return "homo"
    return "negativ"

# 8. Liste aller Spalten (Antigene) im verwendeten DataFrame
alle_spalten = negative_liss.columns.tolist()

# 9. Alleinig stehende Antigene identifizieren (Spalten, die nicht zu einem definierten Paar gehören)
paare_flach = set([x for pair in antigen_paare for x in pair])
alleinstehende_antigene = [ag for ag in alle_spalten if ag not in paare_flach]

# 10. Hauptlogik der Antigenanalyse (Regel 1, 2a & 2b)
gestrichene_antigene_liste = []
for idx, row in negative_liss.iterrows():
    gestrichen = []
    
    # Regel 1: Überprüfe alleinstehende Antigene
    for col in alleinstehende_antigene:
        if row[col] == "+":
            gestrichen.append(col)
    
    # Regel 2a und 2b: Paarweise Auswertung
    for a1, a2 in antigen_paare:
        val1 = row.get(a1, "0")
        val2 = row.get(a2, "0")
        status = zygotie_status(val1, val2)
    
        if status == "homo":
            if row[a1] == "+":
                gestrichen.append(a1)
            if row[a2] == "+":
                gestrichen.append(a2)
        elif status == "hetero":
            for antigen in (a1, a2):
                if antigen in erlaubte_hetero_streichung and row[antigen] == "+":
                    gestrichen.append(antigen)
    # Duplikate entfernen
    gestrichen = list(set(gestrichen))
    gestrichene_antigene_liste.append(gestrichen)

# 11. Ergebnis-DataFrame für gestrichene Antigene (Regel 1 & 2)
result_df = pd.DataFrame({
    "Spender": spender_ids,
    "Gestrichene Antigene (mit Regeln 1 & 2a/2b)": gestrichene_antigene_liste
})
#print(result_df)

# 12. Regel 3: Globale Bewertung – 
# Ein Antigen wird wie folgt bewertet:
#   - Bei ≥3 positiven Testzellen: "Bestätigt (3x +)"
#   - Bei genau 2 positiven Testzellen: "Bestätigt (2x +)"
#   - Bei genau 1 positiven Testzelle: "Nicht ausgeschlossen (1x +)"
#   - Sonst: "Keine Reaktion"

# Update: Positive LISS-Stufen (nun inklusive "+/-")
positive_liss_stufen = ["+/-","1+", "2+", "3+", "4+"]
positive_df = df[df["LISS"].isin(positive_liss_stufen)].reset_index(drop=True)

alle_antigene = df.drop(columns=["Spender", "Spez. Antigen", "Gen.", "LISS"]).columns.tolist()

# 12.1 Globale gestrichene Antigene (aus den negativen Testzellen)
alle_gestrichene = set([ag for sublist in gestrichene_antigene_liste for ag in sublist])
globale_kandidaten = [ag for ag in alle_antigene if ag not in alle_gestrichene]

# 12.2 Für jedes nicht gestrichene Antigen: Anzahl der positiven Reaktionen zählen
antigen_status_liste = []
for antigen in globale_kandidaten:
    count = sum(positive_df[antigen] == "+")
    if count >= 3:
        status = "Bestätigt (3x +)"
    elif count == 2:
        status = "Bestätigt (2x +)"
    elif count == 1:
        status = "Nicht ausgeschlossen (1x +)"
    else:
        status = "Keine Reaktion"
    antigen_status_liste.append({"Antigen": antigen, "Status": status, "Positive Reaktionen": count})
    
regel3_global_df = pd.DataFrame(antigen_status_liste)
#print(regel3_global_df)

# 13. Ableitung eines globalen Status für jedes Antigen (für die Farbcodierung)
antigen_columns = [col for col in df.columns if col not in ['Spender', 'Spez. Antigen', 'Gen.', 'LISS']]

global_status = {}  # Schlüssel: Antigen, Wert: Kategorie
for antigen in antigen_columns:
    if antigen in alle_gestrichene:
        global_status[antigen] = "Ausgestrichen"
    elif antigen in regel3_global_df["Antigen"].values:
        raw_status = regel3_global_df.loc[regel3_global_df["Antigen"] == antigen, "Status"].iloc[0]
        # Hier vereinheitlichen wir die Bezeichnungen:
        if "Bestätigt" in raw_status:
            global_status[antigen] = "Bestätigt"
        elif "Nicht ausgeschlossen" in raw_status:
            global_status[antigen] = "Nicht ausgeschlossen"
        elif "Keine Reaktion" in raw_status:
            global_status[antigen] = "Keine Reaktion"
        else:
            global_status[antigen] = "Keine Reaktion"
    else:
        global_status[antigen] = "Keine Reaktion"

# 14. Definition der Farben pro Kategorie (anpassbar)
color_map = {
    "Bestätigt": "lightgreen",
    "Nicht ausgeschlossen": "yellow",
    "Keine Reaktion": "lightgrey",
    "Ausgestrichen": "red"
}

# 15. Funktion zum Erstellen eines Stylesheets für den DataFrame
def highlight_antigens(dataframe):
    # Erstelle ein leeres DataFrame mit demselben Index und Spalten, das die Style-Strings enthält.
    styles = pd.DataFrame("", index=dataframe.index, columns=dataframe.columns)
    for col in dataframe.columns:
        if col in global_status:
            cat = global_status[col]
            color = color_map.get(cat, "")
            styles[col] = f"background-color: {color}"
    return styles



df.index = df.index + 1
# 16. Anwenden des Styles auf den ursprünglichen DataFrame
styled_df = df.style.apply(highlight_antigens, axis=None)

styled_df


,spendernummer,Spender,D,C,E,c,e,CW,K,k,KpA,KpB,JsA,JsB,FyA,FyB,Jka,Jkb,Lea,Leb,P1,M,N,S,s,LuA,LuB,XgA,Gen.,Spez. Antigen,LISS
1,1,251718,+,+,0,0,+,+,0,+,0,+,nt,nt,+,0,+,0,0,+,+,+,+,0,+,0,+,+,nan,nan,4+
2,2,456531,+,+,0,0,+,0,+,+,0,+,nt,nt,0,+,0,+,0,0,+,+,+,0,+,0,+,0,nan,nan,4+
3,3,547641,+,0,+,+,0,0,0,+,0,+,nt,nt,0,+,+,+,0,+,+,+,0,+,0,0,+,0,nan,nan,4+
4,4,5830,0,+,0,+,+,0,0,+,0,+,nt,nt,+,0,+,0,0,+,0,0,+,+,+,0,+,0,nan,nan,-
5,5,824734,0,0,+,+,+,0,0,+,0,+,nt,nt,0,+,0,+,+,0,0,+,0,+,0,0,+,+,nan,Yt(b+)*,4+
6,6,314242,0,0,0,+,+,0,+,+,0,+,nt,nt,0,+,+,0,0,0,+,0,+,0,+,0,+,+,nan,nan,-
7,7,803065,0,0,0,+,+,0,0,+,+,+,nt,nt,+,0,0,+,0,+,+,+,0,0,+,0,+,0,nan,HLA+*,4+
8,8,399620,+,0,0,+,+,0,0,+,0,+,nt,nt,0,0,+,+,0,+,+,+,+,0,+,0,+,0,nan,nan,4+
9,9,514266,0,0,0,+,+,0,0,+,0,+,nt,nt,0,+,+,0,0,+,0,+,0,+,0,+,+,+,nan,nan,4+
10,10,5885,0,0,0,+,+,0,0,+,0,+,nt,nt,+,0,+,0,0,+,+,0,+,+,0,0,+,+,nan,nan,-
